In [1]:
from bs4 import BeautifulSoup
import logging
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

In [2]:
link_template = 'https://dblp.org/db/conf/sigmod/sigmod2024c.html'
def get_title_author(link):
    resp = requests.get(link, proxies=None)
    soup = BeautifulSoup(resp.content, 'html.parser')
    uls = soup.find_all('ul', class_='publ-list')
    titles, authors = [], []
    for ul in uls[2:]:  # 遍历所有 ul
        articles = ul.find_all('li', class_='entry inproceedings')  # 找到当前 ul 下的所有 li
        print(f"Found {len(articles)} articles in one ul.")
        for li in articles:
            title = li.find('span', class_='title').text
            author_list = [a.text for a in li.find_all('span', itemprop='name')]

            titles.append(title)
            authors.append(", ".join(author_list[:-1]))

    return titles, authors

In [3]:
titles, authors = get_title_author(link_template)
print(titles, authors)
print(titles[0], authors[0])

Found 6 articles in one ul.
Found 6 articles in one ul.
Found 5 articles in one ul.
Found 5 articles in one ul.
Found 5 articles in one ul.
Found 5 articles in one ul.
Found 15 articles in one ul.
Found 14 articles in one ul.
Found 2 articles in one ul.
Found 13 articles in one ul.
Found 11 articles in one ul.
['Apache Arrow DataFusion: A Fast, Embeddable, Modular Analytic Query Engine.', 'Unified Query Optimization in the Fabric Data Warehouse.', 'Measures in SQL.', "ByteCard: Enhancing ByteDance's Data Warehouse with Learned Cardinality Estimation.", 'Automated Multidimensional Data Layouts in Amazon Redshift.', 'Automated Clustering Recommendation With Database Zone Maps.', 'Similarity Joins of Sparse Features.', 'FinSQL: Model-Agnostic LLMs-based Text-to-SQL Framework for Financial Analysis.', 'Rock: Cleaning Data by Embedding ML in Logic Rules.', 'Data-Juicer: A One-Stop Data Processing System for Large Language Models.', 'The Hopsworks Feature Store for Machine Learning.', 'COSMO

In [4]:
print(len(titles), len(authors))


87 87


In [5]:
def graph_mask(title):
    t = title.lower()
    conditions = ['graph neural networks', 'gnns', 'gnn', 'graph', 'graph learning', 'graph embedding', 'network embedding']
    return np.any([c in t for c in conditions])

In [6]:
with open(f'2024-SIGMOD.txt', 'w', encoding='utf8') as fw:
    for t, n in zip(titles, authors):
        if graph_mask(t):
            fw.write(f'1. **{t}**\n\n')
            fw.write(f'    *{n}*\n\n')